In [68]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline

import requests

from pprint import pprint
from datetime import datetime as dt
from datetime import timedelta

In [2]:
# plotly.tools.set_credentials_file(username='ralfsantacruz', api_key='m43Oz35O5mKYCxDDJoDq')

In [3]:
def num_hours_ago(hours):
    """
    Returns UTC time x hours ago.
    """
#     from datetime import datetime as dt
    date = dt.utcnow() - timedelta(hours=hours)
    return dt.strftime(date, "%Y-%m-%dT%H:%M:%S")

In [4]:
def get_coords(location):
    """
    Returns coordinates of address for given input.
    """
    from config import g_api_key
    base_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={location}&key={g_api_key}"
    
    data = requests.get(base_url).json()
    
    try:
        loc = data['results'][0]['geometry']['location']
        
        lat = loc['lat']
        lng = loc['lng']
    except:
        return "404"
    
    return lat,lng

In [66]:
def pollution(lat,lng):
    """
    Returns pollution data from coordinates.
    """
    from config import b_api_key 
    
    end_datetime = num_hours_ago(1)
    start_datetime = num_hours_ago(168)
    
    try:
        base_url = f"https://api.breezometer.com/air-quality/v2/historical/hourly?lat={lat}&lon={lng}&key={b_api_key}&start_datetime={start_datetime}&end_datetime={end_datetime}"
        data = requests.get(base_url).json()
        return data
    except:
        return None

In [65]:
def generate_plot(city):  
    try:    
        lat,lng = get_coords(city)
    except ValueError:
        return "Error: We could not find the coordinates for the entry you supplied."

    pollution_data = pollution(lat,lng)
    
    if pollution_data != None:
        try:
            x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
            y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]

            text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
            text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

            test = [f"Dominant pollutant: {x}\n{y}" for x in text1 for y in text2]

            data = [go.Scattergl(x=x,
                               y=y, 
                               mode="markers", 
                               marker=dict(
                                size=12,
                                color = y, #set color equal to a variable
                                colorscale='Viridis',
                                showscale=True
                               ),  
                               name="AQI",
                                text= test)]

            layout = go.Layout(title=f"AQI for {city.title()}",
                            xaxis=dict(title='Date'),
                            yaxis=dict(title='AQI'))

            fig = go.Figure(data=data, layout=layout)
            
            return py.plot(fig, filename='jupyter-basic_line',auto_open=False)
        
        except:
            return "404"        

# Calling functions to plot:

In [52]:
# x,y = get_coords("beijing")

In [53]:
# print(x)
# pollution_data = pollution(x,y)

In [86]:
# generate_plot("san francisco")

# x = [x['datetime'] for x in pollution_data['data'] if x['data_available'] == True]
# y = [y['indexes']['baqi']['aqi'] for y in pollution_data['data'] if y['data_available'] == True]
# text1 = [text['indexes']['baqi']['dominant_pollutant'] for text in pollution_data['data'] if text['data_available'] == True]
# text2 = [text['indexes']['baqi']['category'] for text in pollution_data['data'] if text['data_available'] == True]

# test = [f"Dominant pollutant: {x}\n{y}" for x,y in zip(text1,text2)]

# data = [go.Scattergl(x=x,
#                    y=y, 
#                    mode="markers", 
#                    marker=dict(
#                     size=10,
#                     color = y, #set color equal to a variable
#                     colorscale='Viridis',
#                     showscale=True,
#                     reversescale=True
#                    ),  
#                    name="AQI",
#                     text= test)]

# layout = go.Layout(title="AQI for beijing",
#                 xaxis=dict(title='Date'),
#                 yaxis=dict(title='AQI'))

# fig = go.Figure(data=data, layout=layout)

# # py.iplot(fig, filename='jupyter-basic_line')

# p = plotly.offline.plot(fig, include_plotlyjs=False, output_type='div')

In [93]:
# print(p)
# print(text1)
# print(text2)
# print(test)
# x,y = "404"
# print(p)